<a href="https://colab.research.google.com/github/Harshil-p02/NLP-chess/blob/master/Copy_of_seq2seq_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install chess

     |████████████████████████████████| 147 kB 12.1 MB/s 


In [ ]:
%cd /content/drive/MyDrive/NLP-Chess_lang-model

/content/drive/MyDrive/NLP-Chess_lang-model


In [ ]:
!ls -l

In [ ]:
!chmod +x /content/drive/MyDrive/NLP-Chess_lang-model/stockfish_13_linux_x64_avx2

In [ ]:
!nvidia-smi
import torch
torch.cuda.is_available()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
from torchtext.legacy import data

SOS_WORD = "<sos>"
EOS_WORD = "<eos>"

SRC = Field(init_token=SOS_WORD, eos_token=EOS_WORD)
TRG = Field(init_token=SOS_WORD, eos_token=EOS_WORD)

train_data, val_data, test_data = data.TabularDataset.splits(path='/content/drive/MyDrive/NLP-Chess_lang-model/8',
															 train="train-split.csv", validation="validate-split.csv",
															 test="test-split.csv", format="csv", fields=[("src", SRC),
																									("trg", TRG)])
# SRC.build_vocab(train_data.src)
# TRG.build_vocab(train_data.trg)
SRC.build_vocab(train_data.src, train_data.trg, val_data.src, val_data.trg, test_data.src, test_data.trg)
TRG.build_vocab(train_data.src, train_data.trg, val_data.src, val_data.trg, test_data.src, test_data.trg)

In [ ]:
print(SRC.vocab.itos[00:30])
print(TRG.vocab.itos[10:30])

In [ ]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 97
Unique tokens in target (en) vocabulary: 97


In [ ]:
batch_size = 64
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Group similar length text sequences together in batches.
train_dataloader, valid_dataloader,test_dataloader = data.BucketIterator.splits(

                              # Datasets for iterator to draw data from
                              (train_data, val_data, test_data),

                              batch_size = batch_size,
                              sort = False,
                              device = device
                              )

# Print number of batches in each split.
print('Created `train_dataloader` with %d batches!'%len(train_dataloader))
print('Created `valid_dataloader` with %d batches!'%len(valid_dataloader))
print('Created `test_dataloader` with %d batches!'%len(test_dataloader))
print(f'batchsize = {batch_size}')

Created `train_dataloader` with 2276 batches!
Created `valid_dataloader` with 127 batches!
Created `test_dataloader` with 127 batches!
batchsize = 64


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [ ]:
class Move_Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [ ]:
class Commentary_Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, move_decoder, commentary_decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.move_decoder = move_decoder
        self.commentary_decoder = commentary_decoder
        self.device = device
        
        assert encoder.hid_dim == move_decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.hid_dim == commentary_decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == move_decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        assert encoder.n_layers == commentary_decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        t = 1
        #insert input token embedding, previous hidden and previous cell states
        #receive output tensor (predictions) and new hidden and cell states
        output, hidden, cell = self.move_decoder(input, hidden, cell)
         
        #place predictions in a tensor holding predictions for each token
        outputs[t] = output
         
        #decide if we are going to use teacher forcing or not
        teacher_force = random.random() < teacher_forcing_ratio
         
        #get the highest predicted token from our predictions
        top1 = output.argmax(1) 
         
        #if teacher forcing, use actual next token as next input
        #if not, use predicted token
        input = trg[t] if teacher_force else top1
        t += 1

        # till move ending token
        while t < trg_len and trg[t-1] != "]":
          #insert input token embedding, previous hidden and previous cell states
          #receive output tensor (predictions) and new hidden and cell states
          output, hidden, cell = self.move_decoder(input, hidden, cell)
          
          #place predictions in a tensor holding predictions for each token
          outputs[t] = output
          
          #decide if we are going to use teacher forcing or not
          teacher_force = random.random() < teacher_forcing_ratio
          
          #get the highest predicted token from our predictions
          top1 = output.argmax(1) 
          
          #if teacher forcing, use actual next token as next input
          #if not, use predicted token
          input = trg[t] if teacher_force else top1
          t += 1

        # now generate commentary
        while t < trg_len:
          output, hidden, cell = self.commentary_decoder(input, hidden, cell)
          outputs[t] = output
          teacher_force = random.random() < teacher_forcing_ratio
          top1 = output.argmax(1) 
          input = trg[t] if teacher_force else top1
          t += 1

        return outputs

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 256
N_LAYERS = 1
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec1 = Move_Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
dec2 = Commentary_Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
# device = 'cpu'
model = Seq2Seq(enc, dec1, dec2, device).to(device)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(100, 256)
    (rnn): LSTM(256, 256, dropout=0.2)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (move_decoder): Move_Decoder(
    (embedding): Embedding(101, 256)
    (rnn): LSTM(256, 256, dropout=0.2)
    (fc_out): Linear(in_features=256, out_features=101, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (commentary_decoder): Commentary_Decoder(
    (embedding): Embedding(101, 256)
    (rnn): LSTM(256, 256, dropout=0.2)
    (fc_out): Linear(in_features=256, out_features=101, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,708,234 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters(),lr=0.01)

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 50
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_dataloader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seperated_move.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

In [ ]:
model.load_state_dict(torch.load('seperated_move.pt'))

test_loss = evaluate(model, valid_dataloader, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 1.381 | Test PPL:   3.979 |


In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()
        
    tokens = sentence
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)])
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)

        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
    trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
    
    for i in range(max_len):
    
        with torch.no_grad():
            output, hidden, _ = model.move_decoder(trg_tensor, hidden, encoder_outputs)

        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        # stopping prediction at ']' ---> move ending
        if pred_token == trg_field.vocab.stoi[']']:
            break

    for i in range(max_len):
                
        with torch.no_grad():
            output, hidden, _ = model.commentary_decoder(trg_tensor, hidden, encoder_outputs)

        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        # stopping prediction at ']' ---> move ending
        if pred_token == trg_field.vocab.stoi['<eos>']:
            break
      
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:]

In [ ]:
import asyncio
import copy

import chess.engine
import chess.pgn
import chess
import re
from tqdm import tqdm

# e4 c5 2. c3 Nc6 3. d4 cxd4 4. cxd4 e6 5. Nf3 Bb4+ 6. Nc3 d6 7. Bd2 Nge7 8. d5 exd5
# src = ['e4', 'c5', 'c3', 'Nc6', 'd4', 'cxd4', 'cxd4', 'e6', 'Nf3', 'Bb4+', 'Nc3', 'd6', 'Bd2', 'Nge7', 'd5']
# trg = ['Rad8']
# res = ['exd5', '<eos>']


def stockfish14_scores(move_list, prediction):

    score = []
    board_in = chess.Board()
    for move in move_list:
      # illegal move in inital game seq
      try:
        board_in.push_san(move)
      except ValueError:
        return score
    # use the input board with initial game state

    engine = chess.engine.SimpleEngine.popen_uci("/content/drive/MyDrive/NLP-Chess_lang-model/stockfish_13_linux_x64_avx2")
    # Scoring ----> Mate(-0) < Mate(-1) < Cp(-50) < Cp(200) < Mate(4) < Mate(1) < MateGiven

    # using copy to have two diff boards for evaluation
    eval_board = copy.deepcopy(board_in)

    info = engine.analyse(eval_board, chess.engine.Limit(time=0.1))
    score.append(info["score"].white().wdl().expectation())
    # win-draw-lose percentage from white's perspective 
    try:
        board_in.push_san(prediction[0])
    except ValueError:
        # print("Illegal move")
        score.append(float('-inf'))
    else:
        info = engine.analyse(board_in, chess.engine.Limit(time=0.1))
        score.append(info['score'].white().wdl().expectation())

    result = engine.play(eval_board, chess.engine.Limit(time=0.1))
    eval_board.push(result.move)
    info = engine.analyse(eval_board, chess.engine.Limit(time=0.1))
    score.append(info['score'].white().wdl().expectation())

    engine.quit()
    return score

# pred = ['err']
# score = stockfish14_scores(src, res)
# +ve score --------> stockfish better
# if score == inf  ------> illelagl move predicted
# (score)

[]

In [ ]:
len(test_data.examples)

8089

In [ ]:
# Stockfish Evaluation
from tqdm import tqdm

stockfish_evaluations = []
for example in tqdm(test_data.examples):  
  move_list = []
  input_ = example.src

  for j in range(len(input_)):
    if input_[j] == '[':
      move = ''
      while input_[j+1] != ']':
        move += input_[j+1]
        j += 1
      move_list.append(move)
  # print(move_list)

  target = example.trg
  
  # print(f'src = {input_}')
  # print(f'trg = {target}')
  
  pred_out = (translate_sentence(input_, SRC, TRG, model, device))
  # print(pred_out)
  pred = ['err']
  for i in range(len(pred_out)):
    if pred_out[i] == '[':
      pred = pred_out[i+1:-1]

  # print(pred)
  prediction = [''.join(pred)]
  # print(prediction)
  stockfish_evaluations.append(stockfish14_scores(move_list, prediction))
print(stockfish_evaluations)
# train with whole moves
# train with harshil's input method with 2 decoders. one for piece and one for co-ords

 19%|█▊        | 1506/8089 [28:35<2:05:05,  1.14s/it]/usr/local/lib/python3.7/dist-packages/chess/__init__.py:3550: RuntimeWarning: coroutine 'stockfish14_scores.<locals>.main' was never awaited
  self._attacked_for_king(king_path | king, self.occupied ^ king) or
 93%|█████████▎| 7507/8089 [2:22:38<11:03,  1.14s/it]

In [ ]:
base = "/content/drive/MyDrive/NLP-Chess_lang-model"
with open(base + "/" + "scores.txt", 'w') as f:
  for score in tqdm(stockfish_evaluations):
    f.write(score)
    f.write("\n")

In [ ]:
  # below is the code for creating stockfish games
  # IGNORE

In [ ]:
import asyncio
import chess
import chess.engine
import chess.pgn as pgn
import multiprocessing as mp
import time
from tqdm import tqdm


base = "/content/drive/MyDrive/NLP-Chess_lang-model"
base1 = "/content/drive/MyDrive/NLP-Chess_lang-model/stockfish_moves_"

pgn_data = open("/content/drive/MyDrive/NLP-Chess_lang-model/Caissabase_PGN.pgn")
boards = pgn.read_game(pgn_data)


def stockfish_player(file):
    game_list = []
    file = str(file)
    global boards
    games = open(base + "/" + file + "_caissabase_data.txt")

    t0 = time.time()
    t1 = t0
    t2 = t0
    for i in tqdm(range(80000)):
        # read a game
        game = games.readline().split()
        board = boards.board()
        # white moves first.
        stockfish_moves_list = []
        original_moves = game

        # if i < 45000:
        #    continue

        # take the game till mid point
        for j in range(len(original_moves)//2):
            board.push(chess.Move.from_uci(original_moves[j]))
            stockfish_moves_list.append(str(original_moves[j]))
        # use stockfish

        async def main() -> None:
            transport, engine = await chess.engine.popen_uci("/content/drive/MyDrive/NLP-Chess_lang-model/stockfish_13_linux_x64_avx2")
            while not board.is_game_over():
                result = await engine.play(board, chess.engine.Limit(time=0.1, depth=18))
                board.push(result.move)
                stockfish_moves_list.append(str(result.move))

            await engine.quit()
        asyncio.set_event_loop_policy(chess.engine.EventLoopPolicy())
        asyncio.run(main())
        game_list.append(stockfish_moves_list)
        # convt move list to string and write in a file


        if (i+1) % 1000 == 0:
            with open(base1 + file + "_" + "0" + "-" + str(i+1) + ".txt", "w") as f:
                for move_list in game_list:
                    str1 = " "
                    str1 = str1.join(move_list)
                    f.write(str1)
                    f.write("\n\n")
            t2 = time.time()
            print(f"Process {file}: 0-{i+1} games in {t2-t1}")
            t1 = t2


if __name__ == "__main__":
    NUM_PROCESSES = 2
    offset = 8

    jobs = []
    for i in range(NUM_PROCESSES):
        jobs.append(i+offset)
    print(jobs)

    pool = mp.Pool(NUM_PROCESSES).map(stockfish_player, jobs)